# Reading Financial Data With Pandas-Datareader

In [1]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import pandas_datareader as pdr
import sys
import os

sys.path.append('../..')

from app.models import stocks
from app import db, create_app

## Read financial data for starting stocks for the last 10 years

In [2]:
date_ref = datetime(2022, 5, 22)
start = date_ref - timedelta(days=365*10)
symbols = [x for x, _ in stocks.starting_stocks]

df = pdr.yahoo.daily.YahooDailyReader(symbols=symbols, start=start).read()
df

Attributes   Adj Close                                                 \
Symbols             FB        AAPL       SONY        MSFT        TTWO   
Date                                                                    
2012-05-24   33.029999   17.262812  13.760000   23.735321   11.510000   
2012-05-25   31.910000   17.170284  13.300000   23.727154   11.560000   
2012-05-29   28.840000   17.475042  13.550000   24.135395   11.900000   
2012-05-30   28.190001   17.685738  13.030000   23.955774   11.730000   
2012-05-31   29.600000   17.641768  13.240000   23.833302   11.520000   
...                ...         ...        ...         ...         ...   
2022-05-16  200.039993  145.539993  86.360001  260.892365  110.110001   
2022-05-17  202.619995  149.240005  87.690002  266.200012  123.080002   
2022-05-18  192.240005  140.820007  87.070000  254.080002  123.050003   
2022-05-19  191.289993  137.350006  88.709999  253.139999  119.760002   
2022-05-20  193.539993  137.589996  89.940002  252.559998  116.120003   

Attributes                               Close                         ...  \
Symbols           NFLX        SNOW          FB        AAPL       SONY  ...   
Date                                                                   ...   
2012-05-24   10.038571         NaN   33.029999   20.190001  13.760000  ...   
2012-05-25   10.031429         NaN   31.910000   20.081785  13.300000  ...   
2012-05-29    9.920000         NaN   28.840000   20.438213  13.550000  ...   
2012-05-30    9.620000         NaN   28.190001   20.684643  13.030000  ...   
2012-05-31    9.062857         NaN   29.600000   20.633215  13.240000  ...   
...                ...         ...         ...         ...        ...  ...   
2022-05-16  186.509995  144.509995  200.039993  145.539993  86.360001  ...   
2022-05-17  190.559998  145.809998  202.619995  149.240005  87.690002  ...   
2022-05-18  177.190002  137.759995  192.240005  140.820007  87.070000  ...   
2022-05-19  183.479996  146.820007  191.289993  137.350006  88.709999  ...   
2022-05-20  186.350006  141.490005  193.539993  137.589996  89.940002  ...   

Attributes        Open                               Volume               \
Symbols           TTWO        NFLX        SNOW           FB         AAPL   
Date                                                                       
2012-05-24   11.740000   10.217143         NaN   50237200.0  496230000.0   
2012-05-25   11.500000   10.085714         NaN   37149800.0  328507200.0   
2012-05-29   11.660000   10.151429         NaN   78063400.0  380508800.0   
2012-05-30   11.750000    9.782857         NaN   57267900.0  529429600.0   
2012-05-31   11.710000    9.528571         NaN  111639200.0  491674400.0   
...                ...         ...         ...          ...          ...   
2022-05-16  109.809998  193.300003  155.440002   27112600.0   86643800.0   
2022-05-17  119.879997  189.169998  151.350006   24872700.0   78336300.0   
2022-05-18  121.279999  186.720001  142.449997   23960000.0  109742900.0   
2022-05-19  122.410004  178.050003  140.039993   24446900.0  136095600.0   
2022-05-20  120.889999  185.869995  148.000000   31437700.0  137194600.0   

Attributes                                                            
Symbols          SONY        MSFT        TTWO        NFLX       SNOW  
Date                                                                  
2012-05-24  2733400.0  52575000.0   2662800.0  20742400.0        NaN  
2012-05-25  1784700.0  29507200.0   1755400.0  33371100.0        NaN  
2012-05-29   994100.0  37758800.0   2372200.0  31997000.0        NaN  
2012-05-30  1624900.0  41585500.0   2075700.0  32890900.0        NaN  
2012-05-31  1219700.0  39134000.0   2719500.0  53803400.0        NaN  
...               ...         ...         ...         ...        ...  
2022-05-16   546100.0  32550900.0   4691200.0  16145000.0  5964500.0  
2022-05-17   562300.0  28828800.0   8128000.0   9876700.0  7513000.0  
2022-05-18  1066600.0  31356000.0   41

In [3]:
app = create_app('dev')
ctx = app.app_context()
ctx.push()

In [4]:
outdir = '../../data'


if not os.path.exists(outdir):
    os.mkdir(outdir)

# Set up app_context
app = create_app()
app_context = app.app_context()
app_context.push()
    
for sym in symbols:
    tmp = (df.xs(sym, axis=1, level=1)
           .dropna()
           .reset_index()
           .assign(symbol=sym)
           .rename(columns=lambda x: x.replace(' ', '_').lower()))
    
    # Save csvs
    print(f'Saving {sym:<4} ... to {outdir}/{sym}.csv')
    tmp.to_csv(f'{outdir}/{sym}.csv', index=False)

    # Attempt saving to db
    try:
        print(f'Adding {sym} to db... ', end='')
        stock_prices = [stocks.StockPrice(**d) for d in tmp.to_dict(orient='records')]
        db.session.add_all(stock_prices)
        db.session.commit()
        print('Done')
        print()

    except Exception as e:
        print('Failed')
        print(e)
        print()

app_context.pop()

Saving FB   ... to ../../data/FB.csv
Adding FB to db... Done

Saving AAPL ... to ../../data/AAPL.csv
Adding AAPL to db... Done

Saving SONY ... to ../../data/SONY.csv
Adding SONY to db... Done

Saving MSFT ... to ../../data/MSFT.csv
Adding MSFT to db... Done

Saving TTWO ... to ../../data/TTWO.csv
Adding TTWO to db... Done

Saving NFLX ... to ../../data/NFLX.csv
Adding NFLX to db... Done

Saving SNOW ... to ../../data/SNOW.csv
Adding SNOW to db... Done

